# Run quantum accurate geometry optimisation on a small molecule

This notebook shows how to geometry optimise a small molecule using RIMP2 accurate gradients.

In [57]:
import os
from glob import glob
from pathlib import Path
import json

import rush

## 0) Setup

In [23]:
# Define our project information
DESCRIPTION = "rush-py qm geo opt notebook"
TAGS = ["qdx", "rush-py-v2", "demo", "qm_geo_opt"]
WORK_DIR = Path.home() / "qdx" / "qm_geo_opt"

In [24]:
# |hide
if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=True)

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(WORK_DIR / ".rush", exist_ok=True)
import sys
os.chdir(WORK_DIR)

In [25]:
os.makedirs(PROTEIN_PDB_FOLDER_PATH, exist_ok=True)

## 0.1) Initialize our rush client and fetch available module paths

In [28]:
# By using the `build_provider_with_functions` method, we will also build helper functions calling each module
client = await rush.build_provider_with_functions(
    access_token=RUSH_TOKEN, url=RUSH_URL, batch_tags=TAGS
)

In [29]:
# |hide
client.workspace = WORK_DIR
client.config_dir = WORK_DIR / ".rush"

## 0.2) Get qm_geo_opt rush module

In [32]:
# Get our latest modules as a dict[module_name, module_path]
# If a lock file exists, load it so that the run is reproducable
# This will be done automatically if you use the `build_provider_with_functions` method
modules = await client.get_latest_module_paths()

In [33]:
module_name = "qm_geo_opt"
module_path = modules[module_name]
print(module_path)

github:talo/qm_geo_opt/2f575ce865e97c1eb5cde26da0265017d95e1940#qm_geo_opt_tengu


In [34]:
help(client.qm_geo_opt)

Help on function qm_geo_opt in module rush.provider:

async qm_geo_opt(*args: [<class 'pathlib.Path'>, dict[str, ~T]], target: rush.graphql_client.enums.ModuleInstanceTarget | None = <ModuleInstanceTarget.NIX_SSH_GPU: 'NIX_SSH_GPU'>, resources: rush.graphql_client.input_types.ModuleInstanceResourcesInput | None = ModuleInstanceResourcesInput(gpus=0, gpu_mem=None, gpu_mem_units=None, cpus=None, nodes=None, mem=None, mem_units=None, storage=10, storage_units=<MemUnits.MB: 'MB'>, walltime=None, storage_mounts=None), tags: list[str] | None = None, restore: bool | None = None) -> [<class 'pathlib.Path'>]
    geometry optimise molecule with quantum accuracy
    
    Module version: github:talo/qm_geo_opt/2f575ce865e97c1eb5cde26da0265017d95e1940#qm_geo_opt_tengu
    
    QDX Type Description:
    
        input: @Conformer;
    
        parameters: {
    
        basis_set:string,
    
        conv_threshold:f64,
    
        max_iterations:u32,
    
        use_internal_coords:bool
    
    

# 1) Run Geometry Optimisation

## 1.0) Create input geometry
We will be creating a qdxf conformer object from an unoptimised cyclobutane xyz

In [69]:
def xyz_to_qdxf_topology(xyz_data : str):
  symbols = []
  geometry = []
  for line in xyz_data.splitlines()[3:]:
    symbol, x, y, z = line.split()
    symbols.append(symbol)
    geometry.extend([float(x), float(y), float(z)])
  return {
    "topology": {
      "symbols": symbols,
      "geometry": geometry
    }
  }

In [70]:
cyclobutane_xyz = """
12
cyclobutane
  C     -0.8058     -0.7366      0.0748
  C     -0.7395      0.8061     -0.0273
  C      0.8059      0.7367     -0.0745
  H     -1.2036      1.2228     -0.9269
  H     -1.1440      1.3387      0.8393
  C      0.7394     -0.8063      0.0271
  H      1.2494      1.1143     -1.0010
  H      1.3093      1.2267      0.7649
  H      1.1448     -1.3373     -0.8404
  H      1.2032     -1.2252      0.9261
  H     -1.2501     -1.1129      1.0022
  H     -1.3087     -1.2276     -0.7648
"""

In [71]:
cyclobutane_qdxf = xyz_to_qdxf_topology(cyclobutane_xyz)

In [76]:
# write the qdxf file to disk
cyclobutane_qdxf_input_file = Path(WORK_DIR) / "cyclobutane.qdxf.json"
with open(cyclobutane_qdxf_input_file, "w") as f:
  json.dump(cyclobutane_qdxf, f)

## 1.1) Create geo opt parameters 

In [77]:
params = {
  "use_internal_coords": True,
  "max_iterations": 100,
  "conv_threshold": 1e-4,
  "basis_set": "cc-pVDZ"
}

## 1.1) Call geometry optimisation over input geometry

In [93]:
(geo_opt_out,) = await client.qm_geo_opt(cyclobutane_qdxf_input_file, params, target="NIX_SSH_GPU")
print(await geo_opt_out.get())

2024-01-30 12:03:32,652 - rush - INFO - Argument 3a48b149-7b27-4087-b1c4-28c2e3866b19 is now RESOLVING
2024-01-30 12:03:34,449 - rush - INFO - Argument 3a48b149-7b27-4087-b1c4-28c2e3866b19 is now ADMITTED
2024-01-30 12:04:03,877 - rush - INFO - Argument 3a48b149-7b27-4087-b1c4-28c2e3866b19 is now DISPATCHED


Exception: (<ModuleFailureReason.RUN: 'RUN'>, ModuleInstanceCommonFailureContext(stdout='', stderr='Error: "QM geometry optimisation failed with error: WARNING: OpenMP is set to use only 1 thread(s)!\\nWARNING: Destructing a running timer.\\nWARNING: Destructing a running timer.\\nWARNING: Stream used without final sync. This is likely a bug.\\nWARNING: Destructing a running timer.\\nWARNING: Destructing a running timer.\\nWARNING: Destructing a running timer.\\n--------------------------------------------------------------------------\\nMPI_ABORT was invoked on rank 1 in communicator MPI COMMUNICATOR 3 DUP FROM 0\\nwith errorcode 5.\\n\\nNOTE: invoking MPI_ABORT causes Open MPI to kill all MPI processes.\\nYou may or may not see output from other processes, depending on\\nexactly when Open MPI kills them.\\n--------------------------------------------------------------------------\\nterminate called after throwing an instance of \'std::runtime_error\'\\n  what():  Error running comman